In [1]:
import pandas as pd

# Research Categories

In [ ]:
# Path
csv_path2020 = "/Users/mariejatsun/Desktop/MDA Project/cordis-h2020projects-csv/euroSciVoc.csv"
csv_path2024 = "/Users/mariejatsun/Desktop/MDA Project/cordis-HORIZONprojects-csv/euroSciVoc.csv"

# Load CSV's 
df_2020 = pd.read_csv(csv_path2020, delimiter=";", on_bad_lines="skip")
df_2024 = pd.read_csv(csv_path2024, delimiter=";", on_bad_lines="skip")

# Look if column euroSciVocPath exists
if 'euroSciVocPath' not in df_2020.columns or 'euroSciVocPath' not in df_2024.columns:
    raise ValueError("Column 'euroSciVocPath' not found")

# Extract category
df_2020['category2020'] = df_2020['euroSciVocPath'].str.split('/').str[1]
df_2024['category2024'] = df_2024['euroSciVocPath'].str.split('/').str[1]

# Count rows per category
category_counts_2020 = df_2020['category2020'].value_counts().reset_index()
category_counts_2024 = df_2024['category2024'].value_counts().reset_index()

# Choose column names
category_counts_2020.columns = ['category', 'count_2020']
category_counts_2024.columns = ['category', 'count_2024']

# Print 
print("\nNumber of projects per category in 2020:")
print(category_counts_2020.to_string(index=False))

print("\nNumber of projects per category in 2024:")
print(category_counts_2024.to_string(index=False))



Number of projects per category in 2020:
                   category  count_2020
           natural sciences       48421
 engineering and technology       25267
medical and health sciences       17468
            social sciences       16024
                 humanities        4028
      agricultural sciences        2985

Number of projects per category in 2024:
                   category  count_2024
           natural sciences       18075
 engineering and technology        6680
medical and health sciences        5985
            social sciences        5027
                 humanities        1739
      agricultural sciences        1283


# Funding Distribution 